In [1]:
%%HTML
<style> code {background-color : pink !important;} </style>

Vehicle Detection using SVMs
===

### Vehicle Detection

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import time
from sklearn.preprocessing import StandardScaler

from default_params import *
from utils import *
from feature_ex import *
from heat import *

%matplotlib inline

In [3]:
#Read image data-set.
# Divide up into cars and notcars
# Read in car and non-car images
def image_file_names (glob_pattern) :
    images = glob.glob (glob_pattern)
    imagelist = []
    for image in images:
        imagelist.append(image)
    return imagelist


cars = []
cars = image_file_names ('../data/vehicles/*/*.png')
notcars = []
notcars = image_file_names ('../data/non-vehicles/*/*.png')

#print ("Cars = {}".format(cars[0:5]))
print ("Cars size = {}".format(len (cars)))
print ("Not Cars size = {}".format(len (notcars)))


car_features = extract_features (cars)
notcar_features = extract_features (notcars)
if (np.shape (car_features[0]) != np.shape (notcar_features[0])) :
    raise 'ERROR: HOG feature vector shapes for cars and not cars is different'
if (np.shape (car_features[0]) != np.shape (notcar_features[0])) :
    raise 'ERROR: COLOR feature vector shapes for cars and not cars is different'
print ("Feature vector size cars = {}".format (np.shape (car_features)))
print ("Feature vector size notcars = {}".format (np.shape (notcar_features)))


Cars size = 8792
Not Cars size = 8968


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Feature vector size cars = (8792, 8460)
Feature vector size notcars = (8968, 8460)


In [4]:
from moredata import *

def useMoreData ():
    morecars, morecarbbs, morenotcars, morenotcarbbs = LoadExtraData ()
    morecar_feats = extract_features_from_frames (morecars, morecarbbs)
    morenotcar_feats = extract_features_from_frames (morenotcars, morenotcarbbs)
    ## Add features from the udacity dataset
    car_features = car_features + morecar_feats
    notcar_features = notcar_features + morenotcar_feats
    return

# 
# useMoreData ()
if (np.shape (car_features[0]) != np.shape (notcar_features[0])) :
    raise 'ERROR: HOG feature vector shapes for cars and not cars is different'
if (np.shape (car_features[0]) != np.shape (notcar_features[0])) :
    raise 'ERROR: COLOR feature vector shapes for cars and not cars is different'
print ("Feature vector size cars = {}".format (np.shape (car_features)))
print ("Feature vector size notcars = {}".format (np.shape (notcar_features)))


Feature vector size cars = (8792, 8460)
Feature vector size notcars = (8968, 8460)


In [5]:
# Data set exploration
def print_min_max (feature_set, name=None) :
    if name == None :
        name =""
    print ("{} Min = {}, Max = {}".format (name, np.min (feature_set), np.max(feature_set)))
print_min_max (car_features, "car_hog_features")
print_min_max (notcar_features, "notcar_hog_features")


car_hog_features Min = 0.0, Max = 4096.0
notcar_hog_features Min = 0.0, Max = 4096.0


In [6]:
from sklearn.preprocessing import StandardScaler
def combine_features (car_features, notcar_features) :
    num_carfeat = len (car_features)
    num_notcarfeat = len (notcar_features)
    X = np.vstack ((car_features, notcar_features)).astype(np.float64)
    y = np.hstack ((np.ones (num_carfeat), np.zeros (num_notcarfeat)))
    print ("All features shape X = {}, shape y = {}".format (np.shape(X), np.shape(y)))
    print_min_max (X, "All Features")    
    return X, y, num_carfeat, num_notcarfeat


#car_features = select_random_subset (SUBSET_SIZE, car_features)
#notcar_features = select_random_subset (SUBSET_SIZE, notcar_features)
#print ("Cars subset size = {}".format(np.shape (car_features)))
#print ("Not Cars subset size = {}".format(np.shape (notcars_features)))

X, y, ncf, nnf = combine_features (car_features, notcar_features)
#SUBSET_SIZE = 1000
#X, y = select_random_subset_data (SUBSET_SIZE, X, y)

def normalize_data (X) :
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)
    return scaled_X, X_scaler

X, X_scaler = normalize_data (X)

print ("Normalized Feature vector size = {}".format (np.shape (X)))
print_min_max (X, "Normalized Features")



All features shape X = (17760, 8460), shape y = (17760,)
All Features Min = 0.0, Max = 4096.0
Normalized Feature vector size = (17760, 8460)
Normalized Features Min = -14.155976671742794, Max = 102.58802116003011


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
print('Using:',DEFAULT_ORIENT,'orientations',DEFAULT_PIX_PER_CELL,
    'pixels per cell and', DEFAULT_CELL_PER_BLOCK,'cells per block')

print('Using spatial binning of:',DEFAULT_SPATIAL_SIZE,
    'and', DEFAULT_HIST_BIN,'histogram bins')

print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')



Using: 9 orientations 8 pixels per cell and 2 cells per block
Using spatial binning of: (32, 32) and 32 histogram bins
Feature vector length: 8460
21.1 Seconds to train SVC...
Test Accuracy of SVC =  0.991
My SVC predicts:  [ 0.  1.  1.  1.  0.  1.  0.  1.  1.  0.]
For these 10 labels:  [ 0.  1.  1.  1.  0.  1.  0.  1.  1.  0.]
0.00459 Seconds to predict 10 labels with SVC


In [8]:
# Save model, scaler
import os
import pickle
from sklearn.externals import joblib
VEHICLE_MODEL_PKL='vehicle_model.pkl'
VEHICLE_SCALER_PKL='vehicle_scaler.pkl'
def rm_models_scaler () :
    try :
        for f in glob.glob(VEHICLE_MODEL_PKL) :
            os.remove (f)
            print ("Removed {}".format(f))
        for f in glob.glob (VEHICLE_SCALER_PKL) :
            os.remove (f)
            print ("Removed {}".format (f))
    except :
        print ("Unable to remove existing models")
def save_models_scaler (svc, scaler) :
    rm_models_scaler () # Remove existing models, scaler
    joblib.dump (svc, VEHICLE_MODEL_PKL)
    joblib.dump (scaler, VEHICLE_SCALER_PKL)
def load_models_scaler () :
    return joblib.load (VEHICLE_MODEL_PKL), joblib.load (VEHICLE_SCALER_PKL)

save_models_scaler (svc, X_scaler)

# To use saved models, uncomment the following line
svc, X_scaler = load_models_scaler()
    
    